In [ ]:
import os
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.data.smr_datamodule import SMR_Data
from src.utils.device import print_data_info
import mne
import bbcpy

In [ ]:
data_dir = Path("D:\\SMR\\")
task_name = "LR"
subject_sessions_dict = {"S5": "all"}
loading_data_mode = "within_subject"
ival = "2s:11s:2ms"
bands = [8, 13]
chans = ['C?,~*3,~*4', 'FC*,~*3,~*4']
# chans = "*"
fallback_neighbors = 4
transform = None
normalize = {"norm_type": "std", "norm_axis": 0}
# normalize = None

process_noisy_channels = True
ignore_noisy_sessions = False

trial_type = "valid"

smr_datamodule = SMR_Data(data_dir=data_dir,
                          task_name=task_name,
                          trial_type=trial_type,
                          subject_sessions_dict=subject_sessions_dict,
                          loading_data_mode=loading_data_mode,
                          ival=ival,
                          bands=bands,
                          chans=chans,
                          fallback_neighbors=fallback_neighbors,
                          transform=transform,
                          normalize=normalize,
                          process_noisy_channels=process_noisy_channels,
                          ignore_noisy_sessions=ignore_noisy_sessions)

subjects_sessions_path_dict = smr_datamodule.collect_subject_sessions(subject_sessions_dict)

In [ ]:
smr_datamodule.prepare_dataloader_1()

# CSP

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from sklearn.model_selection import KFold

print(__doc__)

# #############################################################################
# # Set parameters and read data

# avoid classification of evoked responses by using epochs that start 1s after
# cue onset.
tmin, tmax = -1.0, 4.0
event_id = dict(Left=0, Right=1)
train_data = smr_datamodule.train_data

# Create an events array from train_data.y and event times
events = []
for i, label in enumerate(train_data.y):
    events.append([i, 0, label])

# Convert the events list to a NumPy array
events = np.array(events)
digitization_points = {ch:pos for ch, pos in zip(train_data.chans,train_data.chans.mnt)}

eeg_data = train_data.data

info['dig'] = mne.create_info(ch_names=train_data.chans,
                              sfreq=train_data.fs,
                              ch_types=['eeg'] * train_data.nCh,
                              dig=digitization_points)

# Create an EpochsArray object
epochs = mne.EpochsArray(eeg_data, info, events=events, event_id = event_id)

epochs_train = epochs.copy()
labels = epochs.events[:, -1]

In [ ]:
train_data.chans.mnt

In [ ]:
digitization_points

In [ ]:
epochs_train.get_data().shape

In [ ]:
labels.shape

In [ ]:
# Define a monte-carlo cross-validation generator (reduce variance):
scores = []
epochs_data = epochs.get_data()
epochs_data_train = epochs_train.get_data()

cv = KFold(n_splits=6, shuffle=False)
cv_split = cv.split(epochs_data_train)

# Assemble a classifier
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

# Use scikit-learn Pipeline with cross_val_score function
clf = Pipeline([("CSP", csp), ("LDA", lda)])
scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=None)

# Printing the results
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1.0 - class_balance)
print(
    "Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance)
)

In [ ]:
scores

In [ ]:
scores

In [ ]:
print(
    "Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance)
)

In [ ]:
epochs.info

In [ ]:
# plot CSP patterns estimated on full data for visualization
csp.fit_transform(epochs_data, labels)

In [ ]:
csp.filters_.shape

In [ ]:
csp.plot_filters(epochs.info, scalings=1e-9)

In [ ]:
csp.plot_patterns(epochs.info, ch_type="eeg", units="Patterns (AU)", size=1.5)